In [1]:
#Import Data
from collections import defaultdict
import pandas as pd 
import numpy as np
!pip install openpyxl






In [2]:
HorizantalData = pd.read_excel('../input/dataset/Horizontal_Format.xlsx')

In [3]:
print(HorizantalData)

   TiD        items
0    1  M,O,N,K,E,Y
1    2  D,O,N,K,E,Y
2    3      M,A,K,E
3    4    M,U,C,K,Y
4    5  C,O,O,K,I,E


In [4]:
def generate_frequent_itemset(df, min_support):

    #Split items in the lists
    transactions = df['items'].str.split(',')

    #Store itemsets and support count in dictionary
    item_counts = defaultdict(int)
    total_transactions = len(transactions)

    #Loop on transactions to count item occurrences
    for transaction in transactions:
        unique_items = set(transaction)
        for item in unique_items:
            item_counts[item] += 1


   # --> Check if min_support is a number or a percentage

     #if min_support is a number
    if isinstance(min_support, (int, float)) and not (0 < min_support <= 1):
        min_support_count = min_support

     #if min_support is a ratio/fraction
    elif isinstance(min_support, (int, float)) and (0 < min_support <= 1):
        min_support_count = min_support * total_transactions

     #if min_support is a percentage
    else:
        min_support_count = min_support * total_transactions / 100


    #calculate support and filter frequent items
    frequent_itemsets = {item: support for item, support in item_counts.items() if support >= min_support_count}
    print("Frequent Itemsets With Support >=" , min_support , ":")
    return frequent_itemsets

In [5]:
frequent_itemsets = generate_frequent_itemset(HorizantalData, 3)
frequent_itemsets = dict(sorted(frequent_itemsets.items(), key=lambda x:(-x[1],x[0])))
print(frequent_itemsets)

Frequent Itemsets With Support >= 3 :
{'K': 5, 'E': 4, 'M': 3, 'O': 3, 'Y': 3}


In [6]:
def get_sorted_transactions(frequent_itemsets):
    transactions = HorizantalData['items'].str.split(',')

    T1 = defaultdict(int) #for each transaction
    T2  = []              #to copy values into
    Transactions = []     #array of transactions

    for transaction in transactions:
        items = set(transaction)
        for item in items:
            if frequent_itemsets.get(item) is not None:
                T1.update({item : frequent_itemsets[item]})
        T2 = T1.copy()
        Transactions.append(T2)
        T1.clear()

    Sorted_Transactions = []
    for transaction in Transactions:
        sorted_transactions = sorted(transaction.items(), key=lambda x:(-x[1],x[0]))
        Sorted_Transactions.append(dict(sorted_transactions))

    return Sorted_Transactions

In [7]:
sorted_transactions = get_sorted_transactions(frequent_itemsets)
print(sorted_transactions)

[{'K': 5, 'E': 4, 'M': 3, 'O': 3, 'Y': 3}, {'K': 5, 'E': 4, 'O': 3, 'Y': 3}, {'K': 5, 'E': 4, 'M': 3}, {'K': 5, 'M': 3, 'Y': 3}, {'K': 5, 'E': 4, 'O': 3}]


In [8]:
def get_conditional_pattern_base(frequent_itemsets,sorted_sransactions):

    x = dict(sorted(frequent_itemsets.items(), key=lambda x:(-x[1],x[0])))
    sorted_frequent_itemsets = x.keys() #menna handled this already?
    pattern_base = defaultdict(int)
    temp = defaultdict(int) #used to copy
    coditional_pattern_base = dict()


    for item in sorted_frequent_itemsets:
        for i in range(len(sorted_sransactions)):
            if (sorted_sransactions[i].get(item) is not None):
                transaction = sorted_sransactions[i].keys()
                index = list(transaction).index(item)
                path1 = list(transaction)[:index]
                path = ''
                if(len(path1) > 0):
                    for i in range(len(path1)):
                        path+=path1[i] + ','
                    path = list(path)
                    path.pop()
                    path = ''.join(path)
                pattern_base[path] += 1

        temp = pattern_base.copy()
        coditional_pattern_base[item] = temp
        if('' in coditional_pattern_base[item]):
            coditional_pattern_base[item].pop('')
        pattern_base.clear()
    
    return coditional_pattern_base; 

In [9]:
coditional_pattern_base = get_conditional_pattern_base(frequent_itemsets,sorted_transactions)
frequent_itemsets = dict(sorted(frequent_itemsets.items(), key=lambda x:(-x[1],x[0]), reverse = True))
for item in frequent_itemsets.keys():
    if(dict(coditional_pattern_base[item])):
        print("item:" , item, dict(coditional_pattern_base[item]))

item: Y {'K,E,M,O': 1, 'K,E,O': 1, 'K,M': 1}
item: O {'K,E,M': 1, 'K,E': 2}
item: M {'K,E': 2, 'K': 1}
item: E {'K': 4}


In [10]:
def get_conditional_trees(coditional_pattern_base):
    conditional_trees = dict()
    x = defaultdict(int)

    for item in coditional_pattern_base:
        summ = sum(coditional_pattern_base[item].values())

        if(coditional_pattern_base[item]):
            common = list(coditional_pattern_base[item].keys())[0]
            
            for path in coditional_pattern_base[item]:
                common = set(path) & set(common)
            common = str(common)
            
            for i in range(len(common)):
                if(common[i].isalpha()):
                    x[common[i]] = summ
            temp = x.copy()
            conditional_trees[item] = temp
            x.clear()
            
    return conditional_trees

In [11]:
conditional_trees = get_conditional_trees(coditional_pattern_base)
for item in conditional_trees:
    print(item, dict(conditional_trees[item]))

E {'K': 4}
M {'K': 3}
O {'E': 3, 'K': 3}
Y {'K': 3}
